In [1]:
import torch
from torch.autograd import Variable
import math
import pyro
import pyro.distributions as dist

In [2]:
mu = Variable(torch.zeros(1))
sigma = Variable(torch.ones(1))
x = dist.normal(mu,sigma)
print(x)

Variable containing:
 0.7806
[torch.FloatTensor of size 1]



In [3]:
log_p_x = dist.normal.log_pdf(x,mu,sigma)
print(log_p_x)
print(math.exp(log_p_x.data[0]))

Variable containing:
-1.2236
[torch.FloatTensor of size 1]

0.294167521999


In [5]:
x = pyro.sample("test_sampler", dist.normal,mu,sigma)
print(x)
#can also define your own arbitrary stochastic function and do
# x = pyro.sample("my_sample", fn, arg1, arg2)

Variable containing:
-2.1759
[torch.FloatTensor of size 1]



In [30]:
def weather():
    cloudy = pyro.sample('cloudy',dist.bernoulli,
                        Variable(torch.Tensor([0.3])))
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny'
    mean_temp = {'cloudy':[55.0] , 'sunny':[75.0]}[cloudy]
    sigma_temp = {'cloudy':[10.0] , 'sunny':[15.0]}[cloudy]
    temp = pyro.sample('temp', dist.normal, Variable(torch.Tensor(mean_temp)), Variable(torch.Tensor(sigma_temp)))
    return cloudy,temp.data[0]

('sunny', 67.50688171386719)
('cloudy', 64.01900482177734)
('sunny', 103.29197692871094)


In [37]:
for _ in range(3):
    print (weather())

('cloudy', 52.81415939331055)
('sunny', 58.51919937133789)
('sunny', 101.4261703491211)


# now lets build a larger model from the simpler model

In [38]:
def ice_cream_sales():
    cloudy,temp = weather()
    expected_sales = [200] if cloudy == 'sunny' and temp > 80.0 else [50]
    ice_cream = pyro.sample('ice_cream', dist.normal, 
                            Variable(toch.Tensor(expected_sales)),Variable(torch.Tensor(sigma_temp)))
    return ice_cream

# more complex flows: Recursion, Higher order stochastic, Random Control Flow

In [39]:
def geometric(p, t= None):
    if t is None:
        t = 0
    #in the line below, the format function updates the sample name based on t
    x = pyro.sample("x_{}".format(t), dist.bernoulli,p)
    if torch.equal(x.data, torch.zeros(1)): #hmmm..annoying tensor comparisons
        return x
    else:
        return x + geometric(p, t+1) # stochastic recursion. So if 3 is returned then we "recursed" :-) in thrice

In [49]:
print (geometric(Variable(torch.Tensor([0.5]))))

Variable containing:
 4
[torch.FloatTensor of size 1]

